# Solving MIP

To include Line numbers: select cell -> ctrl + m -> l (= ell) (without ctrl)

Run the cells with code in order; a cell is run by pressing shift+ enter

### Disitrubtion dependence 
The code as written below works for *all* instances. 

However for possible speed-up we've added minor changes for instances from the discrete Uniform distribution. Our changes can be found as comment in the code. These changes need not be made in order for the code to work on instances form the Uniform distribution. 

In [1]:
using JuMP
using CPLEX

## Solving MIP
For JuMP documentation: https://media.readthedocs.org/pdf/jump/latest/jump.pdf 

For CPLEX documentation: https://github.com/JuliaOpt/CPLEX.jl 



In [2]:
function solveIP(n::Int,d::Int,C::Array{Float32,3})  
    #function solveIP(n::Int,d::Int,m::Int,C::Array{Int64,3})
    
    model = Model(solver=CplexSolver(CPX_PARAM_SCRIND=0, CPX_PARAM_TILIM = 3600)); 

    mi = minimum(C);  # remove this line for Uniform specific
    ma = maximum(C);  # remove this line for Uniform specific
    bigM::Float32 =  ma + 1;  # bigM::Float32 =  m + 1;

    @variable(model,t);
    @variable(model,mi <= u[1:d] <= ma);  # @variable(model,0 <= u[1:d] <= m, Int);
    @variable(model,mi <= l[1:d] <= ma);  # @variable(model,0 <= l[1:d] <= m, Int);
    @variable(model,0 <= x[1:n,1:n] <= 1, Int);

    @objective(model, Min, t);

    @constraints(model, begin     
        Assignment1[i in 1:n], sum(x[i,j] for j = 1:n) == 1
        Assignment2[j in 1:n], sum(x[i,j] for i = 1:n) == 1

        UpperBound[i in 1:n, j in 1:n, k in 1:d], C[i,j,k]*x[i,j] <= u[k]
        LowerBound[i in 1:n, j in 1:n, k in 1:d], l[k] <= C[i,j,k]*x[i,j] + bigM*(1-x[i,j])
        Objective[k in 1:d], u[k] - l[k] <= t
     end);
 
    status = solve(model);
    OPT = getobjectivevalue(model);
    
# determine arg_k
        upper = Float32[mi for k in 1:d];  # upper = Float32[0 for k in 1:d];
        lower = Float32[ma for k in 1:d];  # lower = Float32[2*m for k in 1:d];
       for(z2 in 1:n)
            for(z1 in 1:n)
                if(getvalue(x)[z1,z2] == 1)
                    for k in 1:d
                        if(C[z1,z2,k] <= lower[k])
                            lower[k] = C[z1,z2,k]
                        elseif(C[z1,z2,k] >= upper[k])
                            upper[k] = C[z1,z2,k]
                        end
                    end # end for k
                end
            end # for z1
        end #for z2
        A_max = maximum(upper-lower);
        ArgOpt = find(a->a==A_max, upper-lower);
        print(" ", OPT, " ", sum(ArgOpt)/length(ArgOpt)," ", length(ArgOpt), " ");

end

solveIP (generic function with 1 method)

# choose instance type + run

First call of the function requires more running time, due to compiling.
Hence we first solve a small test instance, before running the actual instances.

In [ ]:
    println("testing + compiling")
    C3 = Float32[1 for i in 1:3, j in 1:3, k in 1:3];
    temp = readdlm(pwd()"/test/Norm1.txt")
    for(j in 1:3)
        for i in 1:3
          for k in 1:3
             C3[i,j,k] = temp[3*(i-1)+j,k];    
          end
        end
    end
    @time solveIP(3,3,C3)


#############  Run all instances Class 1  #########

for m in [100,1000]
    for n in [10,20]  
        println(" ")
        println(" ")
        for d in [2,100,300]
            println("MIP:Uniform", m, "n=", n, "d=", d)
            for iter in 1:10
                #local C2 = Int64[1 for i in 1:n, j in 1:n, k in 1:d];
                local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                temp = readdlm(pwd()"/U(0,"*string(m)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")    
                for( j in 1:n)
                    for i in 1:n
                        for k in 1:d
                            C2[i,j,k] = temp[n*(i-1)+j,k];    
                        end
                    end
                end
                print("Ist", iter)
               #@time solveIP(n,d,m,C2)
                @time solveIP(n,d,C2)
            end
        end
    end
end

for E in [50,500]
    for S in [1,10]
        println(" ")
        println(" ")
        for n in [10,20]
            for d in [2,100,300]
                println("MIP:Normal(",E,",",S,")n=", n, "d=", d)
                for iter in 7:10

                    local C2 = Float32[1 for i in 1:n, j in 1:n, k in 1:d];
                    temp = readdlm(pwd()"/N("*string(E)*","*string(S)*")/n"*string(n)*"d"*string(d)*"/Ist"*string(iter)*".txt")
                    for( j in 1:n)
                        for i in 1:n
                            for k in 1:d
                                C2[i,j,k] = temp[n*(i-1)+j,k];    
                            end
                        end
                    end
                    print("Ist", iter)
                    @time solveIP(n,d,C2)

                end
            end
        end 
    end
end


testing + compiling
 1.1839523315429141 2.0 1   4.393509 seconds (5.23 M allocations: 219.347 MB, 1.72% gc time)
 
 
MIP:Uniform100n=10d=2
Ist1 35.0 2.0 1   0.082887 seconds (3.64 k allocations: 459.891 KB)
Ist2 34.0 2.0 1   0.078427 seconds (3.88 k allocations: 499.344 KB)
Ist3 40.0 1.0 1   0.079777 seconds (3.65 k allocations: 460.047 KB)
Ist4 43.0 2.0 1   0.108754 seconds (3.88 k allocations: 495.750 KB)
Ist5 42.0 1.0 1   0.086817 seconds (3.64 k allocations: 459.813 KB)
Ist6 35.0 1.5 2   0.072198 seconds (3.88 k allocations: 495.531 KB)
Ist7 38.0 1.5 2   0.130965 seconds (3.89 k allocations: 499.938 KB)
Ist8 37.99999999999999 2.0 1   0.103942 seconds (3.88 k allocations: 495.188 KB)
Ist9 38.0 2.0 1   0.122623 seconds (3.88 k allocations: 498.859 KB)
Ist10 41.0 2.0 1   0.117558 seconds (3.88 k allocations: 494.578 KB)
MIP:Uniform100n=10d=100
Ist1 94.0 47.666666666666664 3  15.609827 seconds (181.41 k allocations: 13.625 MB, 0.12% gc time)
Ist2 94.0 39.25 4   8.669603 seconds (181.38